In [1]:
import cv2 as cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import time



In [2]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid2.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]

        differenceHipsList = []
        differenceShouldersList = []
        leftshoulderList = []
        rightshoulderList = []
        lsp=[]
        rsp=[]
        lhp=[]
        rhp=[]
        lefthipList = []
        righthipList = []
        midpointsholder_l=[]
        midpointhip_l=[]
        spinelength_l=[]
        ratio_l=[]
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceHips = np.absolute(righthip.z - lefthip.z)
                    differenceHipsList.append(differenceHips)
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceHipsAverage = np.mean(differenceHipsList)
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)

                except:
                    
                    pass
                    

            

 
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y]),slope_shoulder_l.append(slope_shoulder),slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [3]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid3.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
       
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceHips = np.absolute(righthip.z - lefthip.z)
                    differenceHipsList.append(differenceHips)
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceHipsAverage = np.mean(differenceHipsList)
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)

                except:
                    
                    pass
                    
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y]),slope_shoulder_l.append(slope_shoulder),slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [4]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid4.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
       
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceHips = np.absolute(righthip.z - lefthip.z)
                    differenceHipsList.append(differenceHips)
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceHipsAverage = np.mean(differenceHipsList)
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)

                except:
                    
                    pass
                    

            
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y]),slope_shoulder_l.append(slope_shoulder),slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [5]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid5.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
       
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceHips = np.absolute(righthip.z - lefthip.z)
                    differenceHipsList.append(differenceHips)
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])

                except:
                    
                    pass
                    

            

                differenceHipsAverage = np.mean(differenceHipsList)
                differenceShouldersAverage = np.mean(differenceShouldersList)

                
                slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

                rate_slope_shoulder = slope_shoulder * fps

                rate_slope_shoulder_length = differenceShouldersAverage * fps
                midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                lefthip_poin=np.array([lefthip.x,lefthip.y])
                righthip_poin=np.array([righthip.x,righthip.y])
                midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                midpointhip_p_x = (lefthip.x+righthip.x)/2
                midpointhip_p_y = (lefthip.y+righthip.y)/2
                midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y]),slope_shoulder_l.append(slope_shoulder),slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [6]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid6.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
       
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceHips = np.absolute(righthip.z - lefthip.z)
                    differenceHipsList.append(differenceHips)
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceHipsAverage = np.mean(differenceHipsList)
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)

                except:
                    
                    pass
                    
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y]),slope_shoulder_l.append(slope_shoulder),slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [7]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid7.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
       
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceHips = np.absolute(righthip.z - lefthip.z)
                    differenceHipsList.append(differenceHips)
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceHipsAverage = np.mean(differenceHipsList)
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)

                except:
                    
                    pass

                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y]),slope_shoulder_l.append(slope_shoulder),slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [8]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid8.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
       
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceHips = np.absolute(righthip.z - lefthip.z)
                    differenceHipsList.append(differenceHips)
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceHipsAverage = np.mean(differenceHipsList)
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)

                except:
                    
                    pass
                    
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y]),slope_shoulder_l.append(slope_shoulder),slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [9]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid9.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
       
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceHips = np.absolute(righthip.z - lefthip.z)
                    differenceHipsList.append(differenceHips)
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceHipsAverage = np.mean(differenceHipsList)
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)

                except:
                    
                    pass

                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y]),slope_shoulder_l.append(slope_shoulder),slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [10]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid10.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
       
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceHips = np.absolute(righthip.z - lefthip.z)
                    differenceHipsList.append(differenceHips)
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])
                    
                    differenceHipsAverage = np.mean(differenceHipsList)
                    differenceShouldersAverage = np.mean(differenceShouldersList)
                    slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                    slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

                    rate_slope_shoulder = slope_shoulder * fps

                    rate_slope_shoulder_length = differenceShouldersAverage * fps
                    midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                    midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                    leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                    rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                    lefthip_poin=np.array([lefthip.x,lefthip.y])
                    righthip_poin=np.array([righthip.x,righthip.y])
                    midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                    midpointhip_p_x = (lefthip.x+righthip.x)/2
                    midpointhip_p_y = (lefthip.y+righthip.y)/2
                    midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                    ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                    spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)

                except:
                    
                    pass

                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y]),slope_shoulder_l.append(slope_shoulder),slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [11]:
if __name__ == "__main__":
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    mp_holistic=mp.solutions.holistic
    cap = cv2.VideoCapture('Trainingvideos/vid11.mp4')
    # used to record the time when we processed last frame
    prev_frame_time = 0
 
    # used to record the time at which we processed current frame
    new_frame_time = 0
    # VIDEO FEED
    image_l=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
       
        framecounter = 0
        while cap.isOpened():

            ret, frame = cap.read()
            if not ret:
                break
            if frame is not None:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            

                font = cv2.FONT_HERSHEY_SIMPLEX
                new_frame_time = time.time()
            # Calculating the fps
 
            # fps will be number of frame processed in given time frame
            # since their will be most of time error of 0.001 second
            # we will be subtracting it to get more accurate result
                fps = 1 / (new_frame_time - prev_frame_time)
                prev_frame_time = new_frame_time

            # converting the fps into integer
                fps = int(fps)

            # converting the fps to string so that we can display it on frame
            # by using putText function
                fps_1 = str(fps)

            # putting the FPS count on the frame
                cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



                try:
                    landmarks = results.pose_landmarks.landmark
                    leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                    rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                    differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                    righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                    lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                    differenceHips = np.absolute(righthip.z - lefthip.z)
                    differenceHipsList.append(differenceHips)
                    differenceShouldersList.append(differenceShoulders)
                    leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                    rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                    lefthipList.append([lefthip.x, lefthip.y])
                    righthipList.append([righthip.x, righthip.y])

                except:
                    
                    pass
                    

            

                differenceHipsAverage = np.mean(differenceHipsList)
                differenceShouldersAverage = np.mean(differenceShouldersList)

                
                slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
                slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

                rate_slope_shoulder = slope_shoulder * fps

                rate_slope_shoulder_length = differenceShouldersAverage * fps
                midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
                midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
                leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
                rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
                lefthip_poin=np.array([lefthip.x,lefthip.y])
                righthip_poin=np.array([righthip.x,righthip.y])
                midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
                midpointhip_p_x = (lefthip.x+righthip.x)/2
                midpointhip_p_y = (lefthip.y+righthip.y)/2
                midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])
                
                ratio_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)/np.linalg.norm(lefthip_poin-righthip_poin)
                spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
                framecounter += 1
                
            
            
            #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
            #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
            #From there, we should be able to detect in which phase the subject is in each frame.

                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                        )
                
               
                cv2.imshow('Mediapipe Feed', image)
                #image_l.append(image)
                #print(image.shape)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y]),slope_shoulder_l.append(slope_shoulder),slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)
    image_l=np.array(image_l)
    cap.release()
    #print(midpointhip_l)
    cv2.destroyAllWindows()

In [12]:
data={'Left_Shoulder':lsp,
      'Right_Shoulder':rsp,
      'Left_Hip':lhp,
      'Right_Hip':rhp,
      'Midpoint_Shoulder':midpointsholder_l,
      'Midpoint_Hip':midpointhip_l,
      'Spine_Length':spinelength_l,
      'Ratio_of_shoulder/hip':ratio_l,
      'Slope_shoulder':slope_shoulder_l,
      'Slope_hip':slope_hip_l,
      'Rate_of_Shoulder':rate_slope_shoulder_l,
      'Rate_of_hip':rate_slope_shoulder_length_l
      }
dataF=pd.DataFrame(data)


In [13]:
dataF

,Left_Shoulder,Right_Shoulder,Left_Hip,Right_Hip,Midpoint_Shoulder,Midpoint_Hip,Spine_Length,Ratio_of_shoulder/hip,Slope_shoulder,Slope_hip,Rate_of_Shoulder,Rate_of_hip
0,"[0.4659608006477356, 0.23057140409946442]","[0.5405862927436829, 0.26585352420806885]","[0.46946805715560913, 0.43274515867233276]","[0.5077434182167053, 0.44391128420829773]","[0.5032735466957092, 0.24821246415376663]","[0.4886057376861572, 0.43832822144031525]",0.190681,2.070326,0.441643,0.283854,0.000000,0.000000
1,"[0.4659668207168579, 0.23047849535942078]","[0.5405769348144531, 0.26574358344078064]","[0.47014757990837097, 0.4333319664001465]","[0.5077864527702332, 0.44386452436447144]","[0.5032718777656555, 0.2481110394001007]","[0.48896701633930206, 0.43859824538230896]",0.191024,2.111424,0.441536,0.272853,7.064575,0.376238
2,"[0.46596717834472656, 0.23042462766170502]","[0.5405775308609009, 0.26570138335227966]","[0.4705929756164551, 0.43389639258384705]","[0.5078378319740295, 0.44385963678359985]","[0.5032723546028137, 0.24806300550699234]","[0.4892154037952423, 0.43887801468372345]",0.191332,2.140602,0.441663,0.261386,6.624938,0.335186
3,"[0.46594107151031494, 0.2304207682609558]","[0.5405766367912292, 0.26573750376701355]","[0.4707358479499817, 0.4341742694377899]","[0.5078259706497192, 0.4438766837120056]","[0.5032588541507721, 0.24807913601398468]","[0.48928090929985046, 0.43902547657489777]",0.191457,2.153719,0.441970,0.255857,6.629546,0.285960
4,"[0.46576207876205444, 0.2304605394601822]","[0.5405778884887695, 0.26578229665756226]","[0.47100111842155457, 0.4352239668369293]","[0.5078365206718445, 0.44475874304771423]","[0.503169983625412, 0.24812141805887222]","[0.4894188195466995, 0.4399913549423218]",0.192362,2.174401,0.441093,0.253289,6.175299,0.233760
...,...,...,...,...,...,...,...,...,...,...,...,...
4413,"[0.46891239285469055, 0.2042299509048462]","[0.5457616448402405, 0.2386329025030136]","[0.47970303893089294, 0.43308016657829285]","[0.5176816582679749, 0.4406140446662903]","[0.5073370188474655, 0.2214314267039299]","[0.4986923485994339, 0.43684710562229156]",0.215589,2.174621,0.362032,0.106152,5.068443,0.494596
4414,"[0.46882563829421997, 0.20418788492679596]","[0.5459839105606079, 0.23877233266830444]","[0.4796462655067444, 0.43502187728881836]","[0.5176807641983032, 0.442430317401886]","[0.5074047744274139, 0.2214801087975502]","[0.4986635148525238, 0.4387260973453522]",0.217422,2.182094,0.375132,0.098967,6.377250,0.600545
4415,"[0.46875983476638794, 0.20387375354766846]","[0.54625004529953, 0.23895882070064545]","[0.47960177063941956, 0.4359877407550812]","[0.5176349878311157, 0.4439198672771454]","[0.507504940032959, 0.22141628712415695]","[0.49861837923526764, 0.4399538040161133]",0.218718,2.189433,0.365927,0.098062,5.488905,0.529862
4416,"[0.4687178432941437, 0.20304082334041595]","[0.5464903116226196, 0.23883506655693054]","[0.4793182909488678, 0.4360447824001312]","[0.5174589157104492, 0.44398200511932373]","[0.5076040774583817, 0.22093794494867325]","[0.4983886033296585, 0.4400133937597275]",0.219269,2.197615,0.360662,0.102571,5.770591,0.565109
